# EpidemIA

## Introdução

2019 foi um ano que irá ficar nos arautos da história, principalmente pela grande pandemia de uma nova doença com origem nos países asiáticos, uma evolução da SARS, denominada COVID-19.

A ideia de passar novamente por uma epidemia assusta qualquer pessoa nos dias atuais, e olhando as ações que foram tomadas, a única forma de minimizar os problemas e causas é a rápida identificação e comunicação, ou seja, informação.

Pensando no grande fluxo de dados públicos que poderíamos utilizar e na transparência de dados disponibilizados ao público, seria de fácil acesso para que pudéssemos traçar ligações entre doenças e sintomas aos locais de ocorrência, ou até com informações socioeconômicas, porém, infelizmente os dados que podemos acessar são escassos, e por mais que tenhamos acesso as estas informações do Portal da Saúde, só podemos visualizar dados de estatísticas vitais ou de mortalidade, algumas doenças epidemiológicas são citadas mas não existe a possibilidade de verificar novas doenças ou possíveis epidemias pela falta de padrão e informações mais qualificadas como localização, datas exatas, gênero, idade e etc ...

Com este objetivo, coletando as informações de forma padronizada e com máximo de aproveitamento, utilizaremos a Inteligência Artificial para criar modelos que realizam diagnósticos preliminares com base nos sintomas informados pelo usuário.

## Dados

Os datasets utilizados neste arquivos estão disponibilizados no site [Kaggle](https://www.kaggle.com/plarmuseau/sdsort).



### Importação dos dados

In [3]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity

all_symptoms = pd.read_csv('archives/all_symptoms.csv', ';')
all_diseases = pd.read_csv('archives/all_disease.csv', ';')

### Leitura dos dados e criação dos dataframes

In [4]:
def read_data(filename, separator):
    """ Reads in the dataset, and returns a tuple of a pandas dataframe and a sparse matrix of symptom/disease/severity """
    data = pd.read_csv(filename, separator, usecols=[0,1,2], names=['symptom', 'disease','sev'], skiprows=1)

    data=data.dropna(axis=0, how='any')  #drop nan
    
    # map each disease and symptom to a unique numeric value
    data['symptom'] = data['symptom'].astype("category")
    data['disease'] = data['disease'].astype("category")

    # create a sparse matrix of all the symptoms/severities
    sev = coo_matrix((data['sev'].astype(float),
                       (data['disease'].cat.codes.copy(),
                        data['symptom'].cat.codes.copy())))
    
    return data, sev, data.groupby(['disease']).sev.sum(), data['symptom'].cat.codes.copy()


In [5]:
data, matrix, sum_disease, symptom = read_data('archives/dataset.csv', ';')
data.head()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Utilização do algorítmo de peso Okapi BM25

In [6]:
def bm25_weight(data, K1=1.5, B=0.75):
    """ Weighs each row of the matrix data by BM25 weighting """
    # calculate idf per term (symptom)
    N = float(data.shape[0])
    idf = np.log(N / (1 + np.bincount(data.col)))

    # calculate length_norm per document (disease)
    row_sums = np.squeeze(np.asarray(data.sum(1)))
    average_length = row_sums.sum() / N
    length_norm = (1.0 - B) + B * row_sums / average_length

    # weight matrix rows by bm25
    ret = coo_matrix(data)
    ret.data = ret.data * (K1 + 1.0) / (K1 * length_norm[ret.row] + ret.data) * idf[ret.col]
    return ret

symptom_count = data.groupby('symptom').size()


### Utilização de divisão singular do dataframe

O scipy.sparse.linalg.svds realiza a divisão linear entre as linhas e colunas e demonstra a singularidade entre cada nódulo do dataframe, mais informações [clique aqui](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html)

In [7]:
Ur, Si, VTr = svds(bm25_weight(matrix), k=100)
VTr=pd.DataFrame(VTr)

print("Doenças com nível de singularidade por cada peso ", Ur.shape)
print("---------------")

print("Valores singulares para cada nódulo", Si.shape)
print("---------------")

print("Sintomas com nível de singularidade por cada peso", VTr.shape, "\n", VTr.head())
print("---------------")

print("Quantidade de doenças", sum_disease.shape)
print("Quantidade de sintomas", symptom_count.shape)


Doenças com nível de singularidade por cada peso  (1166, 100)
---------------
Valores singulares para cada nódulo (100,)
---------------
Sintomas com nível de singularidade por cada peso (100, 272) 
         0         1         2         3         4         5         6    \
0  0.014101  0.051303 -0.025409 -0.017391  0.008442  0.028194 -0.000629   
1  0.059150  0.027501 -0.030196  0.130698 -0.009655 -0.014612  0.050925   
2 -0.055753 -0.080138 -0.025759 -0.046833  0.034591  0.009857 -0.005437   
3 -0.076163  0.032048 -0.026198  0.024312 -0.000546 -0.022696 -0.074752   
4 -0.021006 -0.012745  0.004355 -0.001930  0.019102  0.018667 -0.128631   

        7         8         9    ...       262       263       264       265  \
0 -0.043522  0.012414  0.035019  ...  0.018039  0.003366 -0.047162  0.026827   
1 -0.020046  0.003795  0.069810  ...  0.000769  0.014065 -0.045792 -0.021742   
2  0.028634  0.008370 -0.013135  ... -0.009764  0.001649  0.034429 -0.037671   
3  0.015704  0.027511  0.0559

## Cosine Similarity

In [20]:
Disease_df = pd.DataFrame(cosine_similarity(Ur,VTr.T), columns=symptom_count.index, index=list(sum_disease.index)) 
Disease_df.head()

symptom,1,2,3,4,5,6,7,8,9,10,...,290,291,292,293,294,295,301,303,304,306
1,0.205484,0.272114,0.043167,-0.047387,-0.021459,0.015953,0.633309,-0.019115,-0.073327,-0.012857,...,-0.029264,-0.128676,-0.001867,-0.023130,-0.015429,0.002837,-0.018442,-0.040733,-0.045832,0.113079
2,-0.046659,-0.078299,-0.071982,0.063161,0.045799,-0.004549,-0.168562,0.049388,0.049649,-0.014734,...,0.074478,0.079732,0.024173,0.001410,-0.000718,-0.005382,-0.054203,0.022342,-0.022638,-0.059992
3,0.114272,0.381978,0.005659,0.049680,-0.043052,-0.014820,0.091332,-0.013405,-0.059609,0.009341,...,-0.033005,-0.009946,-0.020345,0.013041,0.019639,0.024916,-0.026550,-0.056702,0.056844,0.133652
4,0.017050,0.022067,-0.036084,0.040988,0.001289,0.038791,-0.050136,-0.021242,-0.006880,-0.012500,...,0.009720,-0.006165,-0.050009,-0.007232,0.016348,0.006368,0.022296,-0.005299,0.028985,0.002352
5,-0.034572,-0.030762,-0.051101,-0.086641,-0.025164,0.005332,-0.028281,0.025686,-0.041621,-0.038750,...,-0.017533,0.007516,0.002488,-0.010030,0.002354,0.009506,0.020183,0.000352,0.013323,0.140841


### Demonstração de usabilidade

In [7]:
#Sintoma 35 = Foot pain

print("Sintoma ", all_symptoms[all_symptoms['Symptom_id'] == 35])
print("Top 10 prováveis doênças relacionadas")
print("--------------------------------------")
listProb = Disease_df[35].sort_values(ascending=False).index
for x in listProb[:10]:
    print(x, all_diseases[all_diseases['Disease_id'] == x].Disease_desc.values)

Sintoma      Symptom_id Symptom_desc
33          35    Foot pain
Top 10 prováveis doênças relacionadas
--------------------------------------
1098 ['Heel spur']
1097 ['Flat feet pes planus']
621 ['Puncture wound']
582 ['Plantar wart human papilloma virus infection, foot warts']
581 ['Plantar fasciitis inflammation of tissue at the bottom of the foot']
963 ["Sever's disease calcaneal apophysitis, common cause of heel pain"]
1287 ['Foot sprain']
947 ['Foot fracture broken foot']
323 ['Bunion of big toe hallux valgus']
124 ['Calluses and corns']
